IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M4\FAULT_M4_10


In [3]:
guasto=2

VOLO 34 FAULT 10%

In [4]:
rcou_m4_fault10 = pd.read_csv("RCOU.csv")
rcou_m4_fault10 = rcou_m4_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m4_fault10 = rcou_m4_fault10[((rcou_m4_fault10['C9'] >= potenza) & (rcou_m4_fault10['C10'] >= potenza) & (rcou_m4_fault10['C11'] >= potenza) & (rcou_m4_fault10['C12'] >= potenza) & (rcou_m4_fault10['C13'] >= potenza) & (rcou_m4_fault10['C14']>= potenza))]
rcou_m4_fault10=rcou_m4_fault10.reset_index(drop=True)
display(rcou_m4_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,71672974,1454,1444,1499,1395,1501,1392
1,71717439,1535,1415,1486,1471,1540,1410
2,71727490,1354,1591,1516,1453,1389,1567
3,71738394,1353,1604,1506,1478,1368,1595
4,71751266,1407,1583,1550,1448,1461,1539
...,...,...,...,...,...,...,...
5611,140815408,1599,1404,1593,1412,1559,1455
5612,140825207,1593,1406,1594,1404,1558,1450
5613,140836705,1581,1415,1590,1403,1543,1461
5614,140848285,1558,1426,1516,1474,1439,1547


In [5]:
min=rcou_m4_fault10['TimeUS'][0]
max=rcou_m4_fault10['TimeUS'][len(rcou_m4_fault10)-1]
print(max)
print(min)

140859141
71672974


In [6]:
xkf1_m4_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m4_fault10 = xkf1_m4_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m4_fault10 = xkf1_m4_fault10[((xkf1_m4_fault10['TimeUS'] >= min) & (xkf1_m4_fault10['TimeUS'] <= max	))]
xkf1_m4_fault10 = xkf1_m4_fault10.reset_index(drop=True)
print(xkf1_m4_fault10)

         TimeUS  Roll  Pitch     Yaw
0      71683864 -0.46  -0.40  355.35
1      71694656 -0.10  -0.45  355.36
2      71704946  0.10  -0.47  355.35
3      71715377  0.05  -0.48  355.34
4      71725387 -0.14  -0.40  355.35
...         ...   ...    ...     ...
6282  140813337  0.45   4.06    0.42
6283  140823342  0.49   3.88    0.36
6284  140834825  0.52   3.74    0.39
6285  140846172  0.56   3.73    0.45
6286  140856511  0.63   3.68    0.55

[6287 rows x 4 columns]


In [7]:
att_m4_fault10 = pd.read_csv("ATT.csv")
att_m4_fault10 = att_m4_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m4_fault10 = att_m4_fault10[((att_m4_fault10['TimeUS'] >= min) & (att_m4_fault10['TimeUS'] <= max	))]
att_m4_fault10=att_m4_fault10.reset_index(drop=True)
print(att_m4_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0      71685046     0.07 -0.46      0.18  -0.40  355.26  355.36
1      71695745     0.04 -0.10      0.17  -0.45  355.26  355.37
2      71706645     0.03  0.10      0.16  -0.47  355.26  355.36
3      71716773     0.02  0.05      0.15  -0.48  355.26  355.35
4      71726805     0.00 -0.14      0.14  -0.40  355.26  355.36
...         ...      ...   ...       ...    ...     ...     ...
6282  140814739     1.13  0.45      3.87   4.06    1.35    0.42
6283  140824626     1.13  0.49      3.77   3.88    1.35    0.36
6284  140835923     1.12  0.52      3.70   3.74    1.35    0.39
6285  140847431     1.11  0.56      3.62   3.73    1.34    0.45
6286  140858295     1.09  0.63      3.54   3.68    1.34    0.55

[6287 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m4_fault10)>len(xkf1_m4_fault10):
    to_add=att_m4_fault10[len(xkf1_m4_fault10):]
    att_m4_fault10=att_m4_fault10[:len(xkf1_m4_fault10)]

if len(xkf1_m4_fault10)>len(att_m4_fault10):
    to_add=xkf1_m4_fault10[len(att_m4_fault10):]

for idx,i in enumerate(att_m4_fault10['Roll']):
    if(xkf1_m4_fault10['Roll'][idx] != i):
        att_m4_fault10['Roll'][idx] = ((att_m4_fault10['Roll'][idx] + xkf1_m4_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m4_fault10['Pitch']):
    if(xkf1_m4_fault10['Pitch'][idx] != i):
        att_m4_fault10['Pitch'][idx] = ((att_m4_fault10['Pitch'][idx] + xkf1_m4_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m4_fault10['Yaw']):
    if(xkf1_m4_fault10['Yaw'][idx] != i):
        att_m4_fault10['Yaw'][idx] = ((att_m4_fault10['Yaw'][idx] + xkf1_m4_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m4_fault10=pd.concat([att_m4_fault10,to_add])

print(att_m4_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw      Yaw
0      71685046     0.07 -0.46      0.18  -0.40  355.26  355.355
1      71695745     0.04 -0.10      0.17  -0.45  355.26  355.365
2      71706645     0.03  0.10      0.16  -0.47  355.26  355.355
3      71716773     0.02  0.05      0.15  -0.48  355.26  355.345
4      71726805     0.00 -0.14      0.14  -0.40  355.26  355.355
...         ...      ...   ...       ...    ...     ...      ...
6282  140814739     1.13  0.45      3.87   4.06    1.35    0.420
6283  140824626     1.13  0.49      3.77   3.88    1.35    0.360
6284  140835923     1.12  0.52      3.70   3.74    1.35    0.390
6285  140847431     1.11  0.56      3.62   3.73    1.34    0.450
6286  140858295     1.09  0.63      3.54   3.68    1.34    0.550

[6287 rows x 7 columns]


C:\Users\chiar\AppData\Local\Temp\ipykernel_28596\3465845594.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m4_fault10['Yaw'][idx] = ((att_m4_fault10['Yaw'][idx] + xkf1_m4_fault10['Yaw'][idx])/2)


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m4_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m4_fault10 = esc_0_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m4_fault10 = esc_0_m4_fault10[((esc_0_m4_fault10['TimeUS'] >= min) & (esc_0_m4_fault10['TimeUS'] <= max))]
esc_0_m4_fault10=esc_0_m4_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m4_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m4_fault10 = esc_1_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m4_fault10 = esc_1_m4_fault10[((esc_1_m4_fault10['TimeUS'] >= min) & (esc_1_m4_fault10['TimeUS'] <= max))]
esc_1_m4_fault10=esc_1_m4_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m4_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m4_fault10 = esc_2_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m4_fault10 = esc_2_m4_fault10[((esc_2_m4_fault10['TimeUS'] >= min) & (esc_2_m4_fault10['TimeUS'] <= max))]
esc_2_m4_fault10=esc_2_m4_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m4_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m4_fault10 = esc_3_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m4_fault10 = esc_3_m4_fault10[((esc_3_m4_fault10['TimeUS'] >= min) & (esc_3_m4_fault10['TimeUS'] <= max))]
esc_3_m4_fault10=esc_3_m4_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m4_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m4_fault10 = esc_4_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m4_fault10 = esc_4_m4_fault10[((esc_4_m4_fault10['TimeUS'] >= min) & (esc_4_m4_fault10['TimeUS'] <= max))]
esc_4_m4_fault10=esc_4_m4_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m4_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m4_fault10 = esc_5_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m4_fault10 = esc_5_m4_fault10[((esc_5_m4_fault10['TimeUS'] >= min) & (esc_5_m4_fault10['TimeUS'] <= max))]
esc_5_m4_fault10=esc_5_m4_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m4_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m4_fault10 = imu_0_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m4_fault10 = imu_0_m4_fault10[((imu_0_m4_fault10['TimeUS'] >= min) & (imu_0_m4_fault10['TimeUS'] <= max))]

imu_1_m4_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m4_fault10 = imu_1_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m4_fault10 = imu_1_m4_fault10[((imu_1_m4_fault10['TimeUS'] >= min) & (imu_1_m4_fault10['TimeUS'] <= max))]

imu_2_m4_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m4_fault10 = imu_2_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m4_fault10 = imu_2_m4_fault10[((imu_2_m4_fault10['TimeUS'] >= min) & (imu_2_m4_fault10['TimeUS'] <= max))]

imu_m4_fault10 = pd.concat((imu_0_m4_fault10, imu_1_m4_fault10, imu_2_m4_fault10))
imu_m4_fault10=imu_m4_fault10.groupby(imu_m4_fault10.TimeUS, as_index=False).mean()

display(imu_m4_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,71675210,-0.562366,-0.017601,0.058196,-0.412550,-0.252978,-9.731380
1,71677358,-0.381477,0.089880,0.034272,-0.460087,0.749301,-10.227750
2,71679642,0.398408,0.118838,-0.017116,-0.373241,1.252281,-10.283288
3,71684827,-0.014572,0.018070,0.016195,-0.339925,-0.366561,-9.819399
4,71690467,-0.144416,0.112670,-0.008237,-0.512008,0.856650,-10.545677
...,...,...,...,...,...,...,...
23873,140847239,0.082320,-0.139260,0.056204,1.207292,0.850404,-12.974497
23874,140849891,0.112314,0.057861,0.093426,1.795235,0.927131,-13.250347
23875,140853397,0.226368,-0.063633,0.112182,2.494976,0.727650,-13.495127
23876,140855323,0.233055,-0.229394,0.111900,2.790600,0.246165,-13.841943


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m4_fault10 = pd.merge_ordered(imu_m4_fault10,att_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_0_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_1_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_2_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_3_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_4_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_5_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")
m4_fault10 = pd.merge_ordered(m4_fault10, rcou_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
index_to_remove=[]
for ind in att_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

6287
6874
7458
8027
8603
9194
9767


In [13]:
m4_fault10 = m4_fault10[~m4_fault10.TimeUS.isin(index_to_remove)]
m4_fault10["TimeUS"] = m4_fault10["TimeUS"] - m4_fault10.iloc[0]["TimeUS"]
m4_fault10["TimeUS"] = m4_fault10["TimeUS"].astype(int)
m4_fault10["Guasto"] = guasto

In [14]:
from datetime import timedelta
m4_fault10=m4_fault10.reset_index(drop=True)
m4_fault10['TimeUS'] = pd.to_datetime(m4_fault10['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m4_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.562366,-0.017601,0.058196,-0.412550,-0.252978,-9.731380,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2
1,00:00:00.002148,-0.381477,0.089880,0.034272,-0.460087,0.749301,-10.227750,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2
2,00:00:00.004432,0.398408,0.118838,-0.017116,-0.373241,1.252281,-10.283288,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2
3,00:00:00.009617,-0.014572,0.018070,0.016195,-0.339925,-0.366561,-9.819399,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2
4,00:00:00.015257,-0.144416,0.112670,-0.008237,-0.512008,0.856650,-10.545677,0.07,-0.46,0.18,...,3.65,3611.0,4.13,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23873,00:01:09.172029,0.082320,-0.139260,0.056204,1.207292,0.850404,-12.974497,1.12,0.52,3.70,...,2.21,5907.0,3.11,1581.0,1415.0,1590.0,1403.0,1543.0,1461.0,2
23874,00:01:09.174681,0.112314,0.057861,0.093426,1.795235,0.927131,-13.250347,1.11,0.56,3.62,...,2.21,5907.0,3.11,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2
23875,00:01:09.178187,0.226368,-0.063633,0.112182,2.494976,0.727650,-13.495127,1.11,0.56,3.62,...,2.21,5907.0,3.11,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2
23876,00:01:09.180113,0.233055,-0.229394,0.111900,2.790600,0.246165,-13.841943,1.11,0.56,3.62,...,2.21,5907.0,3.11,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m4_fault10)/350)):
        V1=m4_fault10[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28596\1666606250.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28596\1666606250.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.030864,0.027091,1.591966,0.167231,73.0,72.0,9.600851,7.730280,-0.020033,...,4623.779857,3.807920,0.091549,0.246013,3.819888,123.0,118.0,4.965487,2.390926,2
1,1.0,-0.031074,0.027029,1.603054,0.167087,70.0,90.0,8.178750,3.588582,-0.019839,...,4521.596784,3.808523,0.091415,0.255302,3.820471,126.0,121.0,4.998244,2.924997,2
2,2.0,-0.031659,0.027073,1.590950,0.167328,68.0,86.0,7.139029,3.574083,-0.019784,...,4392.080435,3.809122,0.091282,0.264615,3.821051,127.0,129.0,5.001827,3.575073,2
3,3.0,-0.032090,0.027062,1.590287,0.167378,70.0,69.0,8.303635,6.601871,-0.020089,...,4240.804706,3.809718,0.091149,0.273950,3.821628,126.0,128.0,4.975697,4.270106,2
4,4.0,-0.032351,0.027010,1.599338,0.167273,69.0,68.0,9.621091,8.090992,-0.020253,...,4081.694218,3.810310,0.091017,0.283307,3.822201,129.0,127.0,4.953714,4.919595,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,63.0,-0.043531,0.024974,1.894823,0.163734,86.0,87.0,7.988865,6.638097,-0.006416,...,4312.091074,3.872367,0.105572,0.355611,3.885942,140.0,136.0,19.187477,5.781098,2
64,64.0,-0.043680,0.024923,1.905881,0.163618,86.0,85.0,8.893325,8.784616,-0.006116,...,4202.032177,3.873590,0.105938,0.350891,3.887208,146.0,144.0,18.719828,6.099085,2
65,65.0,-0.043715,0.024863,1.918134,0.163446,82.0,0.0,10.718871,6.166330,-0.005929,...,4073.091435,3.874808,0.106299,0.346204,3.888467,142.0,140.0,18.232525,6.475696,2
66,66.0,-0.043707,0.024804,1.929756,0.163262,84.0,0.0,10.339524,6.316158,-0.005560,...,3963.021967,3.876019,0.106655,0.341553,3.889720,148.0,146.0,17.726777,6.804363,2


In [18]:
path_file = path_file.replace(r"M4\FAULT_M4_10", "")
os.chdir(path_file)
df_merged.to_csv('m4_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
